In [ ]:
import numpy as np
from gekko import GEKKO
import pandas as pd

Initialization of Data Parameters

In [ ]:
production = {
    1: {'Spinach': 500, 'Tomatoes': 650, 'Carrots': 400},
    2: {'Spinach': 450, 'Tomatoes': 580, 'Carrots': 400},
    3: {'Spinach': 470, 'Tomatoes': 700, 'Carrots': 410}
}
demand = {
    1: {'Spinach': {'A': (100, 200), 'B': (80, 150), 'C': (50, 90)},
        'Tomatoes': {'A': (150, 300), 'B': (100,200), 'C': (80,160)},
        'Carrots': {'A': (50,100), 'B': (70,140), 'C': (40,90)}},

    2: {'Spinach': {'A': (110,190), 'B': (90,150), 'C': (55,95)},
        'Tomatoes': {'A': (160,310), 'B': (120,220), 'C': (100,170)},
        'Carrots': {'A': (60,120), 'B': (70,160), 'C': (40,95)}},

    3: {'Spinach': {'A': (100,180), 'B': (100,140), 'C': (92.5,85)},
        'Tomatoes': {'A': (140,290), 'B': (120,200), 'C': (110,165)},
        'Carrots': {'A': (80,110), 'B': (140,160), 'C': (50,100)}}
}

In [ ]:
cost = {
    1: {
        'Spinach': {'A': 0.30, 'B': 0.35, 'C': 0.45},
        'Tomatoes': {'A': 0.25, 'B': 0.40, 'C': 0.45},
        'Carrots': {'A': 0.20, 'B': 0.25, 'C': 0.45},
    },
    2: {
        'Spinach': {'A': 0.32, 'B': 0.36, 'C': 0.21},
        'Tomatoes': {'A': 0.28, 'B': 0.42, 'C': 0.48},
        'Carrots': {'A': 0.20, 'B': 0.26, 'C': 0.47},
    },
    3: {
        'Spinach': {'A': 0.30, 'B': 0.38, 'C': 0.46},
        'Tomatoes': {'A': 0.26, 'B': 0.39, 'C': 0.46},
        'Carrots': {'A': 0.21, 'B': 0.25, 'C': 0.45},
    }
}



#### Decision Variables
 Decision will be summation of vegetables & Hubs
Representation

In [ ]:
results = {}

for t in range(1,4):
    m = GEKKO(remote=False)
    V = ['Spinach', 'Tomatoes', 'Carrots']
    H = ['A', 'B', 'C']
    # Define variables in one line
    x = {(v, h): m.Var(lb=0) for v in V for h in H} # Create combination of vegetable and Hubs 3*3 variables

   #Production constraints
    [m.Equation(sum(x[v, h] for h in H) <= production[t][v]) for v in V]
    #adding slack constraints for demand
    #slack variables for under and over production
    slack_under = {(v, h): m.Var(lb=0) for v in V for h in H}
    slack_over  = {(v, h): m.Var(lb=0) for v in V for h in H}

    #Demand constraints
    #adding slack variables to the demand constraints to allow for under and over production
    for v in V:
        for h in H:
            m.Equation(x[v, h] + slack_under[v, h] >= demand[t][v][h][0])
            m.Equation(x[v, h] - slack_over[v, h]  <= demand[t][v][h][1])

    penalty = sum(slack_under[v, h]**2 + slack_over[v, h]**2 for v in V for h in H)
    #Objective function+penalty
    #minimize the total cost of production and the penalty for under and over production
    m.Obj(sum(x[v, h] * cost[t][v][h] for v in V for h in H)+1000*penalty)
    m.solve(disp=False)
    total_cost = sum(x[v, h].value[0] * cost[t][v][h] for v in V for h in H)

    print(f"Week {t}: Total Cost = {total_cost}")
    results[t] = {
        'total_cost': total_cost,
        'orders': { (v, h): x[v, h].value[0] for v in V for h in H }
    }


Week 1: Total Cost = 239.49942746899998
Week 2: Total Cost = 271.34945507
Week 3: Total Cost = 316.92445758915


In [ ]:
print(results[1]['orders'][('Spinach', 'A')], results[1]['orders'][('Spinach', 'B')], results[1]['orders'][('Spinach', 'C')])
print(results[1]['orders'][('Tomatoes', 'A')], results[1]['orders'][('Tomatoes', 'B')], results[1]['orders'][('Tomatoes', 'C')])
print(results[1]['orders'][('Carrots', 'A')], results[1]['orders'][('Carrots', 'B')], results[1]['orders'][('Carrots', 'C')])

99.99849999 79.99824999 49.99774999
149.99874999 99.99799999 79.99774999
49.99899999 69.99874999 39.99774999


In [ ]:
results

{1: {'total_cost': 239.49942746899998,
  'orders': {('Spinach', 'A'): 99.99984999,
   ('Spinach', 'B'): 79.99982499,
   ('Spinach', 'C'): 49.99977499,
   ('Tomatoes', 'A'): 149.99987499,
   ('Tomatoes', 'B'): 99.99979999,
   ('Tomatoes', 'C'): 79.99977499,
   ('Carrots', 'A'): 49.99989999,
   ('Carrots', 'B'): 69.99987499,
   ('Carrots', 'C'): 39.99977499}},
 2: {'total_cost': 271.34945507,
  'orders': {('Spinach', 'A'): 109.99983999,
   ('Spinach', 'B'): 89.99981999,
   ('Spinach', 'C'): 54.99989499,
   ('Tomatoes', 'A'): 159.99985999,
   ('Tomatoes', 'B'): 119.99978999,
   ('Tomatoes', 'C'): 99.99975999,
   ('Carrots', 'A'): 59.99989999,
   ('Carrots', 'B'): 69.99986999,
   ('Carrots', 'C'): 39.99976499}},
 3: {'total_cost': 316.92445758915,
  'orders': {('Spinach', 'A'): 99.999849988,
   ('Spinach', 'B'): 99.99980999,
   ('Spinach', 'C'): 88.749885,
   ('Tomatoes', 'A'): 139.99986998,
   ('Tomatoes', 'B'): 119.99980499,
   ('Tomatoes', 'C'): 109.99976999,
   ('Carrots', 'A'): 79.999

In [ ]:
records = []
for week, data in results.items():
    for (v, h), order in data['orders'].items():
        records.append({
            'Week': week,
            'Hub': h,
            'Vegetable': v,
            'Order': order,
            'Total Cost': data['total_cost']
        })
df = pd.DataFrame(records)

In [ ]:
df[(df['Hub']=='C')& (df['Week']==1)].groupby('Vegetable')['Order'].sum()

Vegetable
Carrots     39.999775
Spinach     49.999775
Tomatoes    79.999775
Name: Order, dtype: float64

In [ ]:
df.groupby(['Week','Total Cost']).agg(Count=("Order", "count")).reset_index() #3 hubs and 3 vegetables that's why 9 orders

,Week,Total Cost,Count
0,1,239.494275,9
1,2,271.344551,9
2,3,316.919597,9


### Sensitivity analysis

In [ ]:
demand_modified = {
    1: {'Spinach': {'A': (100, 200), 'B': (80, 150), 'C': (50, 90)},
        'Tomatoes': {'A': (150, 300), 'B': (100,200), 'C': (80,160)},
        'Carrots': {'A': (50,100), 'B': (70,140), 'C': (40,90)}},

    2: {'Spinach': {'A': (100,190), 'B': (90,150), 'C': (55,95)},
        'Tomatoes': {'A': (160,310), 'B': (120,220), 'C': (100,170)},
        'Carrots': {'A': (60,120), 'B': (70,160), 'C': (40,95)}},

    3: {'Spinach': {'A': (100,180), 'B': (100,140), 'C': (92.5,85)},
        'Tomatoes': {'A': (140,290), 'B': (120,200), 'C': (110,165)},
        'Carrots': {'A': (80,110), 'B': (140,160), 'C': (50,100)}}
} #Changed the demand for week 2 Spinach A from (110,190) to (100,190)

In [ ]:
results = {}

for t in range(1,4):
    m = GEKKO(remote=False)
    V = ['Spinach', 'Tomatoes', 'Carrots']
    H = ['A', 'B', 'C']
    # Define variables in one line
    x = {(v, h): m.Var(lb=0) for v in V for h in H} # Create combination of vegetable and Hubs 3*3 variables

   #Production constraints
    [m.Equation(sum(x[v, h] for h in H) <= production[t][v]) for v in V]
    #adding slack constraints for demand
    #slack variables for under and over production
    slack_under = {(v, h): m.Var(lb=0) for v in V for h in H}
    slack_over  = {(v, h): m.Var(lb=0) for v in V for h in H}

    #Demand constraints
    #adding slack variables to the demand constraints to allow for under and over production
    for v in V:
        for h in H:
            m.Equation(x[v, h] + slack_under[v, h] >= demand_modified[t][v][h][0])
            m.Equation(x[v, h] - slack_over[v, h]  <= demand_modified[t][v][h][1])

    penalty = sum(slack_under[v, h]**2 + slack_over[v, h]**2 for v in V for h in H)
    #Objective function+penalty
    #minimize the total cost of production and the penalty for under and over production
    m.Obj(sum(x[v, h] * cost[t][v][h] for v in V for h in H)+10000*penalty)
    m.solve(disp=False)
    total_cost = sum(x[v, h].value[0] * cost[t][v][h] for v in V for h in H)

    print(f"Week {t}: Total Cost = {total_cost}")
    results[t] = {
        'total_cost': total_cost,
        'orders': { (v, h): x[v, h].value[0] for v in V for h in H }
    }


Week 1: Total Cost = 239.49994257825
Week 2: Total Cost = 268.14994546078003
Week 3: Total Cost = 316.92486578095


In [ ]:
results

{1: {'total_cost': 239.49994257825,
  'orders': {('Spinach', 'A'): 99.999984935,
   ('Spinach', 'B'): 79.999982465,
   ('Spinach', 'C'): 49.999977485,
   ('Tomatoes', 'A'): 149.99998738,
   ('Tomatoes', 'B'): 99.99997998,
   ('Tomatoes', 'C'): 79.999977485,
   ('Carrots', 'A'): 49.999989755,
   ('Carrots', 'B'): 69.999987369,
   ('Carrots', 'C'): 39.999977485}},
 2: {'total_cost': 271.34994546121004,
  'orders': {('Spinach', 'A'): 109.99998399,
   ('Spinach', 'B'): 89.999981989,
   ('Spinach', 'C'): 54.999989461,
   ('Tomatoes', 'A'): 159.99998598,
   ('Tomatoes', 'B'): 119.99997899,
   ('Tomatoes', 'C'): 99.99997599,
   ('Carrots', 'A'): 59.999989954,
   ('Carrots', 'B'): 69.999986981,
   ('Carrots', 'C'): 39.99997649}},
 3: {'total_cost': 316.92486249997,
  'orders': {('Spinach', 'A'): 99.999954485,
   ('Spinach', 'B'): 99.999952235,
   ('Spinach', 'C'): 88.7499885,
   ('Tomatoes', 'A'): 139.99995551,
   ('Tomatoes', 'B'): 119.99995185,
   ('Tomatoes', 'C'): 109.99994979,
   ('Carrot

In [ ]:
records = []
for week, data in results.items():
    for (v, h), order in data['orders'].items():
        records.append({
            'Week': week,
            'Hub': h,
            'Vegetable': v,
            'Order': order,
            'Total Cost': data['total_cost']
        })
df_modified = pd.DataFrame(records)

In [ ]:
df_modified[(df_modified['Hub']=='A')& (df_modified['Week']==2)].groupby('Vegetable')['Order'].sum()

Vegetable
Carrots      59.999990
Spinach      99.999984
Tomatoes    159.999986
Name: Order, dtype: float64

In [ ]:
df_modified.groupby(['Week','Total Cost']).agg(Count=("Order", "count")).reset_index()

,Week,Total Cost,Count
0,1,239.499943,9
1,2,268.149945,9
2,3,316.924866,9


In [ ]:
df_modified.head()

,Week,Hub,Vegetable,Order,Total Cost
0,1,A,Spinach,99.999985,239.499943
1,1,B,Spinach,79.999982,239.499943
2,1,C,Spinach,49.999977,239.499943
3,1,A,Tomatoes,149.999987,239.499943
4,1,B,Tomatoes,99.999980,239.499943
